In [1]:
import numpy as np
import cv2
import os
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [1]:
#RUN NEXT 2 CELLS TO OBTAIN IMGS FOR TRAINING AND TEST DATA

In [3]:
backhand_path=r'C:\Users\deCaY\tennis\test\backhand'
backhand_vids=os.listdir(backhand_path)

for vid in backhand_vids:

    vid_name=vid.split('.')[0]
    #print(vid_name)
    cap = cv2.VideoCapture(os.path.join(backhand_path,vid))
    count=0
    while(True):

        ret, frame = cap.read()
        if ret==False:
            break
        file_name_path = backhand_path + '\\' +str(count)+'-'+ str(vid_name) + '.jpg'
        cv2.imwrite(file_name_path, frame)  
        count=count+1

In [5]:
forehand_path=r'C:\Users\deCaY\tennis\test\forehand'
forehand_vids=os.listdir(forehand_path)

for vid in forehand_vids:

    vid_name=vid.split('.')[0]
    #print(vid_name)
    cap = cv2.VideoCapture(os.path.join(forehand_path,vid))
    count=0
    while(True):

        ret, frame = cap.read()
        if ret==False:
            break
        file_name_path = forehand_path + '\\' +str(count)+'-'+ str(vid_name) + '.jpg'
        cv2.imwrite(file_name_path, frame)  
        count=count+1

In [2]:
train_datagen=ImageDataGenerator(rescale=1.0/255.0,horizontal_flip=True)
training_set=train_datagen.flow_from_directory(r'E:\opencv\tennis\training',target_size=(64,64),batch_size=32,class_mode='binary')

Found 25059 images belonging to 2 classes.


In [3]:
test_datagen=ImageDataGenerator(rescale=1.0/255.0)
test_set=test_datagen.flow_from_directory(r'E:\opencv\tennis\test',target_size=(64,64),batch_size=32,class_mode='binary')

Found 3380 images belonging to 2 classes.


In [4]:
cnn=tf.keras.models.Sequential()

cnn.add(tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation='relu',input_shape=(64,64,3)))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2)))

cnn.add(tf.keras.layers.Conv2D(filters=128,kernel_size=(3,3),activation='relu'))
cnn.add(tf.keras.layers.Conv2D(filters=128,kernel_size=(3,3),activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2)))

cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dropout(0.5))

cnn.add(tf.keras.layers.Dense(units=256,activation='relu'))
cnn.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

In [5]:
cnn.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
cnn.fit(x=training_set,validation_data=test_set,epochs=25)

Epoch 1/25


Predicting single image to see how well model performs

In [20]:
training_set.class_indices

{'backhand': 0, 'forehand': 1}

In [34]:
import numpy as np
from keras.preprocessing import image

test_image=image.load_img(r'C:\Users\deCaY\tennis\single\forehand3.jpg',target_size=(64,64))
test_img=image.img_to_array(test_image)
test_img=np.expand_dims(test_img,axis=0)
result=cnn.predict(test_img)

In [35]:
if result[0][0]==0:
    print('Backhand')
else:
    print('Forehand')

Backhand
